## Generate restaurant_transactions.xlsx ##

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Set seed for reproducibility
np.random.seed(42)

# Create date range for the past 2 quarters (6 months)
end_date = datetime(2025, 5, 1)
start_date = end_date - timedelta(days=180)
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Restaurant chains
restaurants = ['PizzaCo', 'BurgerPlace', 'SandwichShop', 'TacoTime']

# Menu items with prices by restaurant
menu_items = {
    'PizzaCo': {
        'Cheese Pizza': 12.99,
        'Pepperoni Pizza': 14.99,
        'Veggie Pizza': 15.99,
        'Meat Lovers Pizza': 17.99,
        'Hawaiian Pizza': 15.99,
        'Breadsticks': 5.99,
        'Chicken Wings': 9.99,
        'Soda': 2.49,
        'Salad': 7.99,
        'Dessert Pizza': 8.99
    },
    'BurgerPlace': {
        'Classic Burger': 7.99,
        'Cheeseburger': 8.99,
        'Bacon Burger': 10.99,
        'Double Burger': 12.99,
        'Chicken Sandwich': 8.99,
        'Veggie Burger': 9.99,
        'Fries': 3.99,
        'Onion Rings': 4.99,
        'Soda': 2.29,
        'Milkshake': 5.99
    },
    'SandwichShop': {
        'Turkey Club': 8.49,
        'Italian Sub': 9.49,
        'Veggie Delight': 7.49,
        'Chicken Bacon Ranch': 9.99,
        'Meatball Sub': 8.99,
        'Soup': 4.99,
        'Chips': 1.99,
        'Cookie': 1.49,
        'Soda': 2.19,
        'Salad': 6.99
    },
    'TacoTime': {
        'Beef Taco': 3.99,
        'Chicken Taco': 4.29,
        'Fish Taco': 4.99,
        'Burrito': 8.99,
        'Quesadilla': 7.99,
        'Nachos': 6.99,
        'Mexican Rice': 2.99,
        'Beans': 2.49,
        'Soda': 2.19,
        'Churros': 3.49
    }
}

# Payment methods
payment_methods = ['Credit Card', 'Debit Card', 'Cash', 'Mobile App', 'Gift Card']

# Order types
order_types = ['Dine-in', 'Takeout', 'Delivery', 'Drive-thru']

# Locations
locations = ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix']

# Customer IDs (for repeat visits)
customer_ids = [f'CUST{i:04d}' for i in range(1, 151)]

# Transaction types
transaction_types = ['Purchase', 'Refund']

# Create empty dataframe
data = []

# Store original transactions for potential refunds
original_transactions = []

# Generate transactions
transaction_id = 10000
for date in date_range:
    # More transactions on weekends
    is_weekend = date.weekday() >= 5
    daily_transactions = np.random.randint(15, 30) if is_weekend else np.random.randint(10, 20)
    
    for _ in range(daily_transactions):
        transaction_id += 1
        
        # Determine if this is a refund (5% chance)
        if len(original_transactions) > 0 and np.random.random() < 0.05:
            # This will be a refund transaction
            transaction_type = 'Refund'
            
            # Pick a random previous transaction to refund (only credit card transactions)
            eligible_transactions = [t for t in original_transactions if t['PaymentMethod'] == 'Credit Card']
            
            if not eligible_transactions:
                # If no eligible transactions, make it a regular purchase
                transaction_type = 'Purchase'
            else:
                # Select a random transaction to refund
                original_tx = np.random.choice(eligible_transactions)
                
                # Create refund 1-7 days after original transaction
                original_date = datetime.combine(original_tx['Date'], datetime.min.time())
                refund_date = original_date + timedelta(days=np.random.randint(1, 8))
                
                # Only process if refund date is within our date range
                if refund_date.date() <= date.date():
                    # Copy most details from original transaction
                    restaurant = original_tx['Restaurant']
                    customer_id = original_tx['CustomerID']
                    location = original_tx['Location']
                    payment_method = 'Credit Card'  # Refunds only on credit card
                    order_type = original_tx['OrderType']
                    items_ordered = original_tx['ItemsOrdered']
                    
                    # Set refund amounts (negative values)
                    subtotal = -original_tx['Subtotal']
                    discount = -original_tx['Discount']
                    tax = -original_tx['Tax']
                    tip = -original_tx['Tip']
                    delivery_fee = -original_tx['DeliveryFee']
                    total = -original_tx['Total']
                    
                    # Reference the original transaction
                    refund_reason = np.random.choice(['Order Error', 'Food Quality', 'Delivery Issue', 'Customer Dissatisfaction'])
                    original_transaction_id = original_tx['TransactionID']
                    
                    # Random hour during business hours
                    hour = np.random.randint(9, 21)
                    minute = np.random.randint(0, 60)
                    order_time = datetime(date.year, date.month, date.day, hour, minute)
                    
                    # Processing time for refunds is usually quick
                    processing_time = np.random.randint(2, 10)
                    
                    # Employee who processed the refund
                    employee_id = f'EMP{np.random.randint(1, 31):03d}'
                    
                    # Weather doesn't really matter for refunds, but include for consistency
                    weather_conditions = ['Sunny', 'Cloudy', 'Rainy', 'Snowy', 'Clear']
                    weather = np.random.choice(weather_conditions, p=[0.4, 0.3, 0.15, 0.05, 0.1])
                    
                    # No promotions on refunds
                    promotion_code = None
                    
                    # Satisfaction might be low (reason for refund)
                    satisfaction = np.random.randint(1, 3) if np.random.random() < 0.5 else None
                    
                    # Add some data entry errors for refunds (to simulate real-world messiness)
                    if np.random.random() < 0.2:
                        # Occasionally make total slightly off from sum of components
                        total = round(total * np.random.uniform(0.98, 1.02), 2)
                    
                    # Add transaction to data
                    data.append({
                        'TransactionID': transaction_id,
                        'CustomerID': customer_id,
                        'Restaurant': restaurant,
                        'Location': location,
                        'Date': date.date(),
                        'Time': order_time.time(),
                        'OrderType': order_type,
                        'PaymentMethod': payment_method,
                        'ItemsOrdered': items_ordered,
                        'Subtotal': round(subtotal, 2),
                        'Discount': round(discount, 2),
                        'Tax': round(tax, 2),
                        'Tip': round(tip, 2),
                        'DeliveryFee': round(delivery_fee, 2),
                        'Total': round(total, 2),
                        'ProcessingTimeMin': processing_time,
                        'EmployeeID': employee_id,
                        'Weather': weather,
                        'PromotionCode': promotion_code,
                        'SatisfactionRating': satisfaction,
                        'TransactionType': transaction_type,
                        'RefundReason': refund_reason,
                        'OriginalTransactionID': original_transaction_id
                    })
                    continue
        
        # If we reach here, it's a regular purchase transaction
        transaction_type = 'Purchase'
        restaurant = np.random.choice(restaurants)
        customer_id = np.random.choice(customer_ids)
        location = np.random.choice(locations)
        payment_method = np.random.choice(payment_methods)
        order_type = np.random.choice(order_types)
        
        # Order time (more orders during lunch and dinner hours)
        hour = np.random.choice(
            [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21],
            p=[0.03, 0.05, 0.07, 0.1, 0.12, 0.11, 0.06, 0.05, 0.06, 0.09, 0.12, 0.08, 0.04, 0.02]
        )
        minute = np.random.randint(0, 60)
        order_time = datetime(date.year, date.month, date.day, hour, minute)
        
        # Generate order items
        num_items = np.random.randint(1, 5)
        menu = menu_items[restaurant]
        items = np.random.choice(list(menu.keys()), size=num_items, replace=True)
        
        # Calculate order details
        items_ordered = ', '.join(items)
        subtotal = sum(menu[item] for item in items)
        
        # Apply random discount occasionally
        discount = 0
        if np.random.random() < 0.15:  # 15% chance of discount
            discount = round(subtotal * np.random.uniform(0.05, 0.2), 2)
        
        # Add tax
        tax_rate = 0.0825
        tax = round((subtotal - discount) * tax_rate, 2)
        
        # Add tip for applicable order types
        tip = 0
        if order_type in ['Dine-in', 'Delivery']:
            tip = round((subtotal - discount) * np.random.uniform(0.1, 0.25), 2)
        
        # Calculate total
        total = subtotal - discount + tax + tip
        
        # Add occasional delivery fee
        delivery_fee = 0
        if order_type == 'Delivery':
            delivery_fee = round(np.random.uniform(2.99, 5.99), 2)
            total += delivery_fee
        
        # Calculate processing time
        if order_type == 'Dine-in':
            processing_time = np.random.randint(15, 45)  # minutes
        elif order_type == 'Takeout':
            processing_time = np.random.randint(10, 25)
        elif order_type == 'Delivery':
            processing_time = np.random.randint(25, 60)
        else:  # Drive-thru
            processing_time = np.random.randint(5, 15)
        
        # Add some employee IDs
        employee_id = f'EMP{np.random.randint(1, 31):03d}'
        
        # Add weather condition (can affect sales patterns)
        weather_conditions = ['Sunny', 'Cloudy', 'Rainy', 'Snowy', 'Clear']
        weather = np.random.choice(weather_conditions, p=[0.4, 0.3, 0.15, 0.05, 0.1])
        
        # Promotion code (occasionally)
        promotion_code = None
        if np.random.random() < 0.2:  # 20% chance of promotion
            promotions = ['BOGO', 'DISCOUNT10', 'FREESIDE', 'LOYALTY', 'NEWCUST']
            promotion_code = np.random.choice(promotions)
        
        # Satisfaction rating (occasionally)
        satisfaction = None
        if np.random.random() < 0.3:  # 30% chance of providing rating
            satisfaction = np.random.randint(1, 6)  # 1-5 rating
        
        # Add placeholder values for refund-specific fields
        refund_reason = None
        original_transaction_id = None
        
        # Add transaction to data
        transaction_data = {
            'TransactionID': transaction_id,
            'CustomerID': customer_id,
            'Restaurant': restaurant,
            'Location': location,
            'Date': date.date(),
            'Time': order_time.time(),
            'OrderType': order_type,
            'PaymentMethod': payment_method,
            'ItemsOrdered': items_ordered,
            'Subtotal': round(subtotal, 2),
            'Discount': round(discount, 2),
            'Tax': round(tax, 2),
            'Tip': round(tip, 2),
            'DeliveryFee': round(delivery_fee, 2),
            'Total': round(total, 2),
            'ProcessingTimeMin': processing_time,
            'EmployeeID': employee_id,
            'Weather': weather,
            'PromotionCode': promotion_code,
            'SatisfactionRating': satisfaction,
            'TransactionType': transaction_type,
            'RefundReason': refund_reason,
            'OriginalTransactionID': original_transaction_id
        }
        
        data.append(transaction_data)
        
        # Store for potential future refunds (only store if it's a purchase)
        if transaction_type == 'Purchase':
            original_transactions.append(transaction_data)

# Create DataFrame
df = pd.DataFrame(data)

# Add some data entry errors (to make cleaning necessary)
# 1. Inconsistent formatting for some restaurant names
for idx in df.sample(frac=0.02).index:  # 2% of rows
    if df.loc[idx, 'Restaurant'] == 'PizzaCo':
        df.loc[idx, 'Restaurant'] = np.random.choice(['Pizza Co', 'Pizza Co.', 'PizzaCo.'])
    elif df.loc[idx, 'Restaurant'] == 'BurgerPlace':
        df.loc[idx, 'Restaurant'] = np.random.choice(['Burger Place', 'BurgerPlace.', 'Burger_Place'])

# 2. Capitalization inconsistencies in payment methods
for idx in df.sample(frac=0.03).index:  # 3% of rows
    payment = df.loc[idx, 'PaymentMethod']
    df.loc[idx, 'PaymentMethod'] = payment.lower() if np.random.random() < 0.5 else payment.upper()

# 3. Occasional missing values
for idx in df.sample(frac=0.01).index:  # 1% of rows
    col = np.random.choice(['Weather', 'EmployeeID', 'ProcessingTimeMin'])
    df.loc[idx, col] = None

# 4. Duplicate transactions (accidentally entered twice)
if len(df) > 10:
    duplicate_count = int(len(df) * 0.01)  # 1% duplicates
    duplicates = df.sample(duplicate_count).copy()
    duplicates['TransactionID'] = range(max(df['TransactionID']) + 1, 
                                       max(df['TransactionID']) + duplicate_count + 1)
    df = pd.concat([df, duplicates])

# Save to Excel file
df.to_excel('restaurant_transactions.xlsx', index=False)

print(f"Excel file created with {len(df)} transactions, including refunds and data quality issues to clean")

Excel file created with 3078 transactions, including refunds and data quality issues to clean
